# Test Case LVV-T2578
This notebook tests the filter changer timing and position repeatability as per https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2578 test case. In particular:

The requirement: LVV-14633 LTS-508-REQ-0015-V-01: Filter Changing_1
* Filter changer must be able to switch between filters remotely
* Positioning requirement is given in requirement LTS-508-4
* Filter change time: 30 s maximum
* Repeatability of motion: < +/- 0.1 mm (TBR)
* Filter installation positioning and reconfiguration: +/- 0.1 mm lateral, +/- 2.6 arcmin for rotation. (verified elsewhere)


In [ ]:
from lsst.sitcom import vandv

exec_info = vandv.ExecutionInfo()
print(exec_info)

## Setup

In [ ]:
import asyncio
import logging
import os
import yaml

import astropy.units as u
import numpy as np
import pandas as pd

from astropy.time import Time
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst_efd_client import EfdClient
from lsst.ts import salobj

Setting up logger

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
logger = logging.getLogger("setup")
logger.level = logging.DEBUG

Instantiate script for logging into EFD and start script task

In [ ]:
logger.info(f'Your UID is {os.getuid()}')
index = os.getuid() * 10 + np.random.randint(0, 9)

logger.info(f'The generated index is {index}')

In [ ]:
test_message = "LVV-T2578 ComCam OptoMechanical Filter Change Test"
script = salobj.Controller("Script", index=index)
await script.start_task

Create EFD client

In [ ]:
client = vandv.efd.create_efd_client()

Make sure DDS Daemon is running and startup Domain

In [ ]:
domain = salobj.Domain()

MTCS initialization

In [ ]:
mtcs = MTCS(domain=domain, log=logger)
mtcs.set_rem_loglevel(10)

In [ ]:
await mtcs.start_task

ComCam initialization

In [ ]:
comcam = ComCam(domain=domain)
comcam.set_rem_loglevel(40)

In [ ]:
await comcam.start_task

In [ ]:
await comcam.enable()

Helper functions to run the filter change sequence and to plot histograms and simple statistics for each filter move.

In [ ]:
async def run_filter_change_sequence(filters, sequence):
    """ This function performs the filter changes sequentially as dictated by the variable `sequence`, 
    records the camera events and saves the move duration and linear encoder position in 
    a dataframe for later analysis.
    
    Parameters
    ----------
    
    filters: list
        Filter list as the output of the command `await comcam.get_available_instrument_setup()`
            
    sequence : list
        List containing the slot numbers of the sequence of filter changes. 
        
    """
    
    move = [] 

    # Move to slot 0 to start sequence
    await comcam.rem.cccamera.cmd_setFilter.set_start(name = filters[0]) 

    for i,val in enumerate(sequence):   
        # Flush events
        comcam.rem.cccamera.evt_startSetFilter.flush()
        comcam.rem.cccamera.evt_endSetFilter.flush()

        # Get start time
        startdate = Time.now()

        # Change Filter command
        await comcam.rem.cccamera.cmd_setFilter.set_start(name = filters[val]) 

        # Record startSetFilter and endSetFilter events
        setFilter = await comcam.rem.cccamera.evt_startSetFilter.next(flush=False, timeout=10)

        endSetFilter = await comcam.rem.cccamera.evt_endSetFilter.next(flush=False, timeout=40)

        # Duration of move based on the startSetFilter and endSetFilter events
        duration = endSetFilter.private_sndStamp - setFilter.private_sndStamp

        logger.info(f'Move to Slot {endSetFilter.filterSlot} from Slot {sequence[i-1]} \t Filter: {endSetFilter.filterName} \t '
                    f'Filter Position Linear Encoder: {endSetFilter.filterPosition} [mm] \t --- \t'
                    f'Duration: {duration:0.3f} [sec]')    

        # Build PD entry for i move
        move.append(
            {
                'ToFilterSlot': endSetFilter.filterSlot, 
                'FromFilterSlot': sequence[i-1],
                'FilterName': endSetFilter.filterName, 
                'FilterPosition': endSetFilter.filterPosition, 
                'Duration': round(duration,3),
            }
        )

        # Get end time
        enddate = Time.now()
        print("Movement "+str(i)+" duration: "+str(24*60*60*(enddate-startdate))+" sec" )

    df = pd.DataFrame(move)
    return df

In [ ]:
def plot(df, column, allInOnePlot=False):
    """Plots the histogram and statistics (mean and std) for each filter move.  
    
    Parameters
    ----------
    
    df: dataframe
        Dataframe generated during the filter change test. 
        
    column : string
        Column from the dataframe generated during this test to plot the histograms. Options are 'Duration' (time to perform a filter change) and
        'FilterPosition' (linear encoder of each filter slot position) 
    
    allInOnePlot : bool
        All histograms in a single plot. Default is False, meaning that each move will be plotted in a histogram separatedly. 
    
    """
    
    units = {'Duration':'[sec]', 'FilterPosition': '[mm]'} 
             
        
    if allInOnePlot:
        plt.figure(figsize=(12,8))
        for k,toSlot in enumerate(sequence[:int(len(sequence)/n)]):
            fromSlot = sequence[k-1]
            move_df = df[(df.ToFilterSlot==toSlot) & (df.FromFilterSlot==fromSlot)]
            move_mean = move_df[column].mean()
            move_std = move_df[column].std()
            move_df[column].hist(bins = 1,
                               label = f'To Slot {toSlot} from Slot {fromSlot} --- mean {move_mean:0.3f}    std {move_std:0.3f} {units[column]}')
        plt.legend()
        plt.xlabel(f'{column} {units[column]}')
        plt.ylabel("Frequency")
        plt.title(f'{column} Histogram')
    
    else:
        fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12,8))
        ax = axes.flatten()

        colors = plt.rcParams["axes.prop_cycle"]()

        for i, toSlot in enumerate(sequence[:int(len(sequence)/n)]):
            c = next(colors)["color"]    
            fromSlot = sequence[i-1]
            move_df = df[(df.ToFilterSlot==toSlot) & (df.FromFilterSlot==fromSlot)]
            move_mean = move_df[column].mean()
            move_std = move_df[column].std()
            move_df[column].hist(ax = ax[i], bins = 10, color = c).set_title(
                f'To Slot {toSlot} from Slot {fromSlot} \n mean {move_mean:0.3f}    std {move_std:0.3f} {units[column]}')
            
            ax[i].set_xlabel(f'{column} {units[column]}')
            ax[i].set_ylabel("Frequency")

        
        fig.subplots_adjust(hspace=1)
        fig.tight_layout()
                   
        
        plt.show() 

Publish to the EFD that LVV-T2578 test is starting 

In [ ]:
script.log.info(f'START- {test_message} -- at {Time.now()}')

## Test

Get available instrument configurations and declare `filters`, number of loop executions `n` and the filter slot changing order `sequence`

In [ ]:
filters = await comcam.get_available_instrument_setup()
logger.info(f'Available filters are {filters}')

In [ ]:
# Number of times to repeat the sequence below
n = 10

In [ ]:
# Declare the filter slot changing order
sequence = [1, 2, 1, 0, 2, 0]*n

----
### Rotator at 0 deg

In [ ]:
script.log.info(f'START- {test_message} 0 degrees -- at {Time.now()}')

In [ ]:
await mtcs.move_rotator(0)

In [ ]:
df_0deg = await run_filter_change_sequence(filters, sequence)
df_0deg.name = '0 deg'

In [ ]:
script.log.info(f'END- {test_message} 0 degrees -- at {Time.now()}')

In [ ]:
plot(df_0deg, 'Duration', allInOnePlot=False)

In [ ]:
plot(df_0deg, 'FilterPosition', allInOnePlot=False)

---
### Rotator at +90 deg

In [ ]:
script.log.info(f'START- {test_message} with MTRotator at +90 degrees -- at {Time.now()}')

In [ ]:
await mtcs.move_rotator(90)

In [ ]:
df_90deg = await run_filter_change_sequence(filters, sequence)
df_90deg.name = '90 deg'

In [ ]:
plot(df_90deg, 'Duration', allInOnePlot=False)

In [ ]:
plot(df_90deg, 'FilterPosition', allInOnePlot=False)

In [ ]:
script.log.info(f'END- {test_message} with MTRotator at +90 degrees -- at {Time.now()}')

---
### Rotator at -90 deg 

In [ ]:
script.log.info(f'START- {test_message} with MTRotator at -90 degrees -- at {Time.now()}')

In [ ]:
await mtcs.move_rotator(-90)

In [ ]:
df_minus90deg = await run_filter_change_sequence(filters, sequence)
df_minus90deg.name = '-90 deg'

In [ ]:
plot(df_minus90deg, 'Duration', allInOnePlot=False)

In [ ]:
plot(df_minus90deg, 'FilterPosition', allInOnePlot=False)

In [ ]:
script.log.info(f'END- {test_message} with MTRotator at -90 degrees -- at {Time.now()}')

---
### Move Duration Summary 

In [ ]:
for df in [df_0deg, df_90deg, df_minus90deg]:
    print(f'\n Rotator angle {df.name} \n')
    for i, toSlot in enumerate(sequence[:int(len(sequence)/n)]):
        fromSlot = sequence[i-1]
        move_df = df[(df.ToFilterSlot==toSlot) & (df.FromFilterSlot==fromSlot)]
        move_mean = move_df['Duration'].mean()
        move_std = move_df['Duration'].std()
        print(f'{toSlot} <- {fromSlot}: mean {move_mean:0.3f} std {move_std:0.3f} [sec]')

---
### Filter Position Repeatability

In [ ]:
def rms(x):
    return np.sqrt((x**2).sum()/len(x))

###!!! Replace commanded positions. Ask Brian
commanded = {'0': -72, '1' : -0.5, '2': 71.5}

for df in [df_0deg, df_90deg, df_minus90deg]:
    print(f'\n Rotator angle {df.name} \n')
    for i, toSlot in enumerate(sequence[:int(len(sequence)/n)]):
        fromSlot = sequence[i-1]
        move_df = df[(df.ToFilterSlot==toSlot) & (df.FromFilterSlot==fromSlot)]
        position_mean = move_df['FilterPosition'].mean()
        position_std = move_df['FilterPosition'].std()
        error_rms = rms(commanded[str(toSlot)]- move_df['FilterPosition'].values)
        print(f'{toSlot} <- {fromSlot}\t mean {position_mean:0.3f} \t  std {position_std:0.3f} \t '
              f' rms commanded - actual  \t {error_rms:0.3f} [mm]')       

## Wrap-up

Announce the EFD that test is done 

In [ ]:
script.log.info(f'END- {test_message} -- at {Time.now()}')

Transition ComCam to STANDBY and close domain. 

In [ ]:
await comcam.standby()

In [ ]:
await domain.close()